In [1]:
from PIL import Image
from pix2tex.cli import LatexOCR
import pytesseract
import easyocr
import numpy as np
import os
import matplotlib.pyplot as plt
import cv2
from collections import deque
import math

In [2]:
class Segmentation_BFS:
    

    def __init__(self, image_path):
        self.image_path = image_path
        pass

    '''Cегментация'''

    #Ищет контур изображения
    def contour(self, tres=(235, 300), show=False):
        img = cv2.imread(self.image_path,  cv2.IMREAD_GRAYSCALE)
        img = cv2.Canny(img, threshold1=tres[0], threshold2=tres[1])
        self.image = img
        if show is True:
            plt.imshow(img, cmap='gray')

        return img

    #Размазывает одну строку/cтолбец (вектор) на precision в обе стороны
    def add_precision(self, vector, precision=1):
        if (precision == 0):
            return vector

        i = 0
        fin = len(vector)

        while True:
            if (vector[i] != 0):
                if ((i != 0) and (vector[i - 1] == 0)):
                    for j in range(max(0, i - precision), i):
                        vector[j] = vector[i]
                if ((i != len(vector) - 1) and (vector[i + 1] == 0)):
                    for j in range(i, min(len(vector), i + precision + 1)):
                        vector[j] = vector[i]
                    i = min(len(vector), i + precision + 1) - 1


            i += 1
            if i >= len(vector) - 1:
                break

        return vector
    
    
    # Получает на  вход 
    # Гиперпараметр pc (precision_coefficient), отвечающий за степень размытия вектора
    # Вектор нумпи, который предстает размыть
    # Размазывает хуету пропорционально ее размерам
    def MARAZM(self, vector, pc=0.5):

        copy = vector.copy()

        ln = len(vector)
        counter = 0


        i = 0
        while i <= ln - 1:
            if copy[i] != 0:
                counter += 1
                i += 1
            else:
                if counter != 0:
                    copy[max(0, math.ceil(i - counter - pc * counter)): min(ln - 1, math.floor(i + pc * counter))] = 1

                    i += math.floor(pc * counter)
                    i = min(ln, i)

                    counter = 0
                else:
                    i += 1

        return copy
    
    # Получает на  вход 
    # Гиперпараметр pc (precision_coefficient), отвечающий за степень размытия вектора
    # Вектор нумпи, который предстает размыть
    def MARAZM_root(self, vector, pc=0.5, ceiling=float('inf'),):

        copy = vector.copy()

        ln = len(vector)
        counter = 0


        i = 0
        while i <= ln - 1:
            if copy[i] != 0:
                counter += 1
                i += 1
            else:
                if counter != 0:
                    copy[max(0, math.ceil(i - counter - pc * math.sqrt(counter)), i - counter - ceiling):\
                         min(ln - 1, math.floor(i + pc * math.sqrt(counter)), i + ceiling)] = 1

                    i += min(math.floor(pc * math.sqrt(counter)), ceiling)
                    i = min(ln, i)

                    counter = 0
                else:
                    i += 1

        return copy

    
    #Итеративно вызывает add_precision() или MARAZM() для каждой строки (если ось х) 
    #или каждого столбца (если ось у), тем самым размазывая изображение
    
    def blur(self, precision=1, ceiling=float('inf'), axis='x', kind = 'MARAZM', show=False):

        t_img = self.image.copy()

        if axis != 'x':
            t_img = t_img.T

        if kind == 'add_precision':
            for row in t_img:
                add_precision(row, precision)
        elif kind == 'MARAZM':
            for i in range(len(t_img)):
                t_img[i] = MARAZM(t_img[i], precision, ceiling=ceiling)
        elif kind == 'MARAZM_root':
            for i in range(len(t_img)):
                t_img[i] = MARAZM_root(t_img[i], precision, ceiling=ceiling)


        if axis != 'x':
            t_img = t_img.T

        if show is True:
            plt.imshow(t_img, cmap='gray')

    return t_img


    #Запускает обход в ширину при обнаружении размазни. Вызывается в следующей функции
    def BFS_segment(self, blurred, been, coords, show=False):

        segment = np.zeros_like(blurred)
        segment[coords[0]][coords[1]] = 1

        q_x = deque()
        q_x.append(coords[1])

        q_y = deque()
        q_y.append(coords[0])

        while q_x:
            i = q_y.popleft()
            j = q_x.popleft()

            if (j - 1 >= 0) and (been[i][j - 1] == 0) and (blurred[i][j - 1] == 1):
                q_x.append(j - 1)
                q_y.append(i)

                been[i][j - 1] = 1
                segment[i][j - 1] = 1

            if (i - 1 >= 0) and (been[i - 1][j] == 0) and (blurred[i - 1][j] == 1):
                q_x.append(j)
                q_y.append(i - 1)

                been[i - 1][j] = 1
                segment[i - 1][j] = 1

            if (j + 1 < len(blurred[0])) and (been[i][j + 1] == 0) and (blurred[i][j + 1] == 1):
                q_x.append(j + 1)
                q_y.append(i)

                been[i][j + 1] = 1
                segment[i][j + 1] = 1

            if (i + 1 < len(blurred)) and (been[i + 1][j] == 0) and (blurred[i + 1][j] == 1):
                q_x.append(j)
                q_y.append(i + 1)

                been[i + 1][j] = 1
                segment[i + 1][j] = 1

        if show is True:
            plt.imshow(segment, cmap='gray')
 
        return segment
       
    #Ищет размазни, встречая их вызывает обход в ширину (BFS_segment)
    def segmentation_BFS(self, blurred):
        been = np.zeros_like(blurred)
        blurred[blurred != 0] = 1
        segments = []
        while not np.array_equal(been, blurred):
            for i in range(len(blurred)):
                for j in range(len(blurred[i])):
                    if (blurred[i][j] != 0) and (been[i][j] == 0):
                        been[i][j] = 1
                        segment = self.BFS_segment(blurred, been, coords=(i, j), show=False)
                        segments.append(segment)
        return segments
       
       
    #Объединяет предыдущие функции для поиска сегментов
    def get_segments_old(self, cont_tres=(500, 600), precision=(6, 4)):
        cv2.imread(self.image_path,  cv2.IMREAD_GRAYSCALE)
        cont_img = self.contour(self.image_path, tres=cont_tres)
        blurred = self.blur(cont_img, precision=precision[0], axis='x')
        blurred = self.blur(blurred, precision=precision[1], axis='y')
        segments = self.segmentation_BFS(blurred)

        return segments
    
    def get_segments(self, cont_tres=(500, 600), precision=(9, 3), ceiling=(10, 5), kind='MARAZM_root'):
        cont_img = self.contour(image_path=self.image_path, tres=cont_tres)
        blurred = self.blur(blurred, precision=1, kind='add_precision')
        blurred = self.blur(blurred, precision=1, axis = 'y', kind='add_precision')
        segments = self.segmentation_BFS(blurred)
        for i in range(len(segments)):
            self.cut_projection(blurred=blurred, segment = segments[i], ret='fill')
        blurred = self.blur(blurred, precision=precision[0], ceiling=ceiling[0], kind=kind)
        blurred = self.blur(blurred, precision=precision[1], ceiling=ceiling[1], kind=kind)
        segments = self.segmentation_BFS(blurred)
        
        return segments

    '''Методы визуализации и получения изображения'''
   
    #Возвращает кусок изображения принадлежащий сегменту
    def show_part_of_image(self, segment, image, show=False):
        mask = segment.copy()
        mask[mask != 0] = 1
        mask = mask.astype(bool)

        iz = np.where(mask.astype(bool), image, 0)
       
        if show is True:
            plt.imshow(iz, cmap='gray')

        return iz
   
    #Возвращает квадрат сегмента и сегмент в нем если ret == 'cut' (в виде маски) либо координаты сегмента
    # если ret == 'coords' или замазывает исходную хуету квадратиком если ret == 'fill'
    def cut_projection(self, segment, blurred, ret = 'cut', show=False):
        x_proj = segment.any(axis=0)
        y_proj = segment.any(axis=1)


        def find_first(proj):
            for i in range(len(proj)):
                if proj[i]!=0:
                    return i

        def find_last(proj):
            for i in range(len(proj) - 1, 0, -1):
                if proj[i]!=0:
                    return i

        y_first = find_first(y_proj)
        y_last = find_last(y_proj)
        x_first = find_first(x_proj)
        x_last = find_last(x_proj) 

        if ret == 'cut':
            cut_segment = segment.copy()[y_first:y_last, x_first:x_last]

            cut_segment[cut_segment!=0] = 255

            if show is True:
                plt.imshow(cut_segment, cmap='gray')

            return cut_segment

        elif ret == 'coords':
            if show is True:
                cut_segment = segment.copy()[y_first:y_last, x_first:x_last]

                cut_segment[cut_segment!=0] = 255

                plt.imshow(cut_segment, cmap='gray')

            return x_first, x_last, y_first, y_last

        elif ret == 'fill':
            blurred[y_first: y_last + 1, x_first:x_last + 1] = 1

            if show is True:
                blurcop = blurred.copy()
                blurcop[blurcop!=0] = 255
                plt.imshow(blurcop, cmap='gray')

        else:
            return None

    
    '''cv2 -> PIL'''
    #Перевод cv2 -> PIL
    def cv2_to_PIL(self, opencv_image):

        color_converted = cv2.cvtColor(opencv_image, cv2.COLOR_BGR2RGB)
        pil_image=Image.fromarray(color_converted)
    
        return pil_image